In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [4]:
html_ = requests.get('https://lhr.nu.edu.pk/faculty/') 
print(html_)
soup = BeautifulSoup(html_.content,'html.parser')



all_deptTags = soup.find_all('a',href= True,class_ = "mt-5 btn btn-nu white", type = "button")
deptNames=[]
dept_abbre=[]
for deptTag in all_deptTags:
    deptNames.append(deptTag.text)
    dept_abbre.append(deptTag['href'].strip('#'))

<Response [200]>


In [5]:
id = []
department = []
name = []
designation = []
HEC_approved = [] 
H_education = []  
email = []
extension = [] 
ImageURL = []


In [6]:



all_faculty = soup.find_all('div', class_ = "container" , id = dept_abbre)
sum =0
for dept in all_faculty:
    
    dt = dept['id'] 
    
    faculty = dept.find_all('div', class_= "facultyCard")
    
    for ftag in faculty: 
        
        try:
            id.append(int(ftag.find('a',class_='faculty-link')['href'].split('/')[-1]))
        except:
            id.append(-1)
        
        name.append(ftag.find('h5',class_ = 'text-center').text)
        
        designation.append(ftag.find('p',class_='small').text.strip().split('\n')[0])
        
        if("HEC Approved" in ftag.find('p',class_='small').text.strip().split('\n')[-1].strip()):
            HEC_approved.append(True)
        else:
            HEC_approved.append(False)    
        
        
        email.append(ftag.find('p', class_ ='mb-0').text)
        
        ImageURL.append('https://lhr.nu.edu.pk'+ftag.find('img', class_='card-img-top')['src'])
        
        urlExt = 'https://lhr.nu.edu.pk'+ftag.find('a',class_='faculty-link')['href']

        data = requests.get(urlExt).content

        soup = BeautifulSoup(data, 'html.parser')

        try:
            extension.append(soup.find('span', class_ = 'small').text.split(':')[1])
        except:
            extension.append(-1)

        p_tag = soup.find('div' , class_= 'col-lg-8 col-md-6 col-sm-12 text-justify')

        try:
            H_education.append(p_tag.find('li').text.split(',')[0].strip())
        except:
            H_education.append('NA')
            
        department.append(dt)

In [8]:
lhr_df = pd.DataFrame({
    'ID':id,
    'Name':name,
    'Designation':designation,
    'HEC Approved':HEC_approved,
    'Highest Education': H_education,
    'Email' : email,
    'Department' : department,
    'Extension' : extension,
    'ImageURL' : ImageURL
})

print(len(id))

184


In [9]:

csv_file_path = 'lhr.csv'
lhr_df.to_csv(csv_file_path, index=False)

In [2]:
import pandas as pd
lhr =pd.read_csv('lhr.csv')
khi =pd.read_csv('karachi.csv')
isb =pd.read_csv('isb.csv')

fast=pd.concat([lhr,khi,isb], ignore_index=True)
fast.to_csv('fast_faculty.csv', index=False)

sample_ = fast.sample(frac=6/10)
sample_.to_csv('sample.csv', index=False)
